<a href="https://colab.research.google.com/github/Nicompag/ISYS5002_portfolio/blob/main/ISYS5002_Assignment_20201867_Nico_Engler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Modules

In [18]:
#!pip install yfinance
#import yfinance as yfin
#yfin.pdr_override()
!pip install kora -q
'''load packages'''
import requests
from bs4 import BeautifulSoup
from kora.selenium import wd
import pandas as pd


     |████████████████████████████████| 57 kB 2.6 MB/s 
     |████████████████████████████████| 56 kB 3.4 MB/s 


#1. Get Input

In [37]:
ticker = input('''Please enter the company's stock market ticker symbol: ''')

# Input validation
#if ticker.isdigit() == True:
#  print('Please enter a valid stock market ticker symbol!')



Please enter the company's stock market ticker symbol: MSFT


#2. Get Data

In [23]:
# Establish Session
url = 'https://au.finance.yahoo.com/'
wd.get(url)

In [39]:
# Work out URL pattern
template = 'https://au.finance.yahoo.com/quote/{ticker1}/profile?p={ticker2}'

url = template.format(ticker1 = ticker, ticker2 = ticker)
wd.get(url)

In [40]:
def get_url(ticker):
  template = 'https://au.finance.yahoo.com/quote/{ticker1}/profile?p={ticker2}'
  url = template.format(ticker1 = ticker, ticker2 = ticker)

  return url

In [41]:
soup = BeautifulSoup(wd.page_source, 'html.parser')

#3. Extract Collection

In [75]:
table = soup.find_all('tr', {'class': 'C($primaryColor) BdB Bdc($seperatorColor) H(36px)'})
table[4].td.text

'Brett  Iversen'

In [51]:
def extract_name(item):
  name = item.td.text
  return name

In [105]:
item = table[0]
salary = item.find('td', 'Ta(end)')
salary = salary.text

salary[-1]

'M'

In [89]:
def extract_salary(item):
  try:
    salary = item.find('td', 'Ta(end)')
    salary = salary.text 
    if salary[-1] == "M":
      salary = float(salary[:-1])*1000000
    else:
      salary = float(salary[:-1])*1000
  except ValueError:
    salary = 'N/A'
  return salary

In [91]:
extract_salary(table[6])

''

In [108]:
def extract_title(item):
  title = item.find('td', 'Ta(start) W(45%)')
  title = title.text
  return title

In [109]:
def extract_record(item):
  return {
      'Name': extract_name(item),
      'Title': extract_title(item),
      'Salary': extract_salary(item)
  }

In [92]:
for i in table:
  print(extract_name(i))

Mr. Satya  Nadella
Mr. Bradford L. Smith LCA
Ms. Amy E. Hood
Ms. Alice L. Jolla
Brett  Iversen
Mr. Frank X. Shaw
Mr. Christopher C. Capossela
Mr. Keith Ranger Dolliver Esq.
Ms. Kathleen T. Hogan
Mr. Ali Mohamed El Faramawy


In [110]:
records = []
for i in table:
  records.append(extract_record(i))

In [111]:
df = pd.DataFrame.from_records(records)
df

,Name,Title,Salary
0,Mr. Satya Nadella,Chairman & CEO,1.36e+07
1,Mr. Bradford L. Smith LCA,"Pres, Chief Legal Officer & Vice Chairman",4.27e+06
2,Ms. Amy E. Hood,Exec. VP & CFO,4.83e+06
3,Ms. Alice L. Jolla,Corp. VP of Fin. & Admin. and Chief Accounting...,
4,Brett Iversen,Gen. Mang. of Investor Relations,
5,Mr. Frank X. Shaw,Corp. VP for Corp. Communications,
6,Mr. Christopher C. Capossela,Exec. VP of Marketing & Consumer Bus. and Chie...,
7,Mr. Keith Ranger Dolliver Esq.,"VP, Deputy Gen. Counsel of Corp., External & L...",
8,Ms. Kathleen T. Hogan,Exec. VP & Chief HR Officer,
9,Mr. Ali Mohamed El Faramawy,Head of The Emerging Markets Digital Transform...,


#4. Calculate Tax

In [122]:
def calculate_tax(income):
  if income <= 18200:
    tax_payable = 0
  elif 18200 < income <= 45000:
    tax_payable = (income - 18200) * 0.19
  elif 45000 < income <= 120000:
    tax_payable = 5092 + 0.325 * (income - 45000)
  elif 120000 < income <= 180000:
    tax_payable = 29467 + 0.37 * (income - 120000)
  elif income > 180000:
    tax_payable = 51667 + 0.45 * (income - 180000)
  
  return tax_payable

In [124]:
df['Expected income tax'] = calculate_tax(df['Salary'])

df

TypeError: ignored